
<div style="text-align:center">
<img src="https://thumbs.gfycat.com/IllSharpCod-max-1mb.gif" width='350'>
</div>

# <span style="color:#ff5f27;">📝 BTC timeseries parsing (600 last days)</span>

In [1]:
# !pip install unicorn-binance-rest-api

In [2]:
from unicorn_binance_rest_api.manager import BinanceRestApiManager as Client
import datetime
import pandas as pd
import os

In [3]:
# !pip install python-dotenv

## API keys are stored in .env file in the next format:
`BINANCE_API_KEY = "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa"`

`BINANCE_API_SECRET = "bbbbbbbbbbbbbbbbbbbbbbbbbbbbbbb"`


`TWITTER_API_KEY = "cccccccccccccccccccccccccccc"`

`TWITTER_API_SECRET = "ddddddddddddddddddddddddddddddddddd"`

### So you need to have .env file with all these API keys inside project directory.

In [4]:
from dotenv import load_dotenv
load_dotenv()

True

In [5]:
def timestamp_2_time(x):
    dt_obj = datetime.datetime.strptime(str(x), '%Y-%m-%d %H:%M:%S')
    dt_obj = dt_obj.timestamp() * 1000
    return int(dt_obj)

In [6]:
def get_client():
    BINANCE_API_KEY = os.getenv("BINANCE_API_KEY")
    BINANCE_API_SECRET = os.getenv("BINANCE_API_SECRET")
    return Client(api_key=BINANCE_API_KEY, api_secret=BINANCE_API_SECRET)


def get_data(since_this_date=None, until_this_date=datetime.datetime.now(), number_of_days_ago=None, crypto_pair="BTCUSDT"):
    client = get_client()
    
    # Calculate the timestamps for the binance api function
    if since_this_date:
        since_this_date += datetime.timedelta(days=1)
    if number_of_days_ago:
        until_this_date = datetime.datetime.now()
        since_this_date = until_this_date - datetime.timedelta(days=number_of_days_ago)
    # Execute the query from binance - timestamps must be converted to strings !
    candle = client.get_historical_klines(crypto_pair, Client.KLINE_INTERVAL_1DAY, str(since_this_date), str(until_this_date))

    # Create a dataframe to label all the columns returned by binance so we work with them later.
    df = pd.DataFrame(candle, columns=['dateTime', 'open', 'high', 'low', 'close', 'volume', 'closeTime', 'quoteAssetVolume', 'numberOfTrades', 'takerBuyBaseVol', 'takerBuyQuoteVol', 'ignore'])
    # as timestamp is returned in ms, let us convert this back to proper timestamps.
    df.dateTime = pd.to_datetime(df.dateTime, unit='ms').dt.strftime("%Y-%m-%d %H:%M:%S")
    df.set_index('dateTime', inplace=True)
    # now returns ALL columns
    return df.drop(['closeTime','ignore'],axis = 1)


def parse_btc_data(last_date=None, number_of_days_ago=None):
    df = get_data(since_this_date=last_date, until_this_date=datetime.datetime.now(),
                  number_of_days_ago=number_of_days_ago + 1)
    df.index.name = 'date'
    df.reset_index(inplace = True)
    df.columns = [*df.columns[:6],'quote_av','trades','tb_base_av','tb_quote_av']
    cols = [*df.columns]
    cols.remove('date')
    cols.remove('trades')
    df[cols] = df[cols].apply(lambda x: x.apply(float))
    df.trades = df.trades.apply(int)
    df['unix'] = pd.to_datetime(df.date).apply(timestamp_2_time)
    return df

## Lets get data for previous 600 days (600 rows)

In [7]:
df = parse_btc_data(number_of_days_ago=600)

C:\Users\rkaha\Documents\hopsworks\venv38\lib\site-packages\dateparser\date_parser.py:35: PytzUsageWarning: The localize method is no longer necessary, as this time zone supports the fold attribute (PEP 495). For more details on migrating to a PEP 495-compliant implementation, see https://pytz-deprecation-shim.readthedocs.io/en/latest/migration.html
  date_obj = stz.localize(date_obj)


In [8]:
df.head(3)

,date,open,high,low,close,volume,quote_av,trades,tb_base_av,tb_quote_av,unix
0,2021-01-02 00:00:00,29331.70,33300.00,28946.53,32178.33,129993.873362,4.073842e+09,2245922,67446.305246,2.110335e+09,1609542000000
1,2021-01-03 00:00:00,32176.45,34778.11,31962.99,33000.05,120957.566750,4.057598e+09,2369698,59750.332871,2.004428e+09,1609628400000
2,2021-01-04 00:00:00,33000.05,33600.00,28130.00,31988.71,140899.885690,4.429010e+09,2642408,69088.469230,2.173435e+09,1609714800000


In [9]:
df.to_csv("data/btc_raw_600.csv")